# Data Migration
**Prerequisites**
- Access to a CDF Project.
- Know how to use a terminal, so you can run `pygen` from the command line to 
  generate the SDK.
- Knowledge of your the data and data models.

In [1]:
import warnings
warnings.filterwarnings('ignore')
# This is just to enable improting the generated SDK from the examples folder in the pygen repository
import sys
from tests.constants import REPO_ROOT
sys.path.append(str(REPO_ROOT / "examples" ))

## Introduction to Problem

In the development of a solution, it is common that you move from one data model to another. Typically, the physicaly storage of data, the containers, stays the same, or are extended, 
and only the views of a data model change. In this case, there is no need for a data migration and all you do is update the solution code for the new model. 

If you, however, need to change how the data is stored in the containers, you need to move the data from one model to the other. A common to do this is to use CDF Transformations. An alternative
ways is to use `pygen` for data migration.

#### Advantage of CDF Transformations
* Can handle large volume of data, millions of instances.
* You can write the transformations in the UI using SQL.


#### Advantages of using `pygen` 
* Edges are automatically created.
* Can handle medium volume of data, hundreds of thousands of instances.
* Support doing very custom migration (what ever you can do in Python code).

**Why** would you **chose `pygen`** over **CDF Transformations**? If you are more comfortable with `Python` than writing `SQL`, you have data models with a of one-to-many edges, the number of nodes and edges (instances) is in the order of hundreds of thousands.




## Use Case

The use case we use in this guide is moving from one Asset Performance Model to another. 

We wil refer to the model we are moving away from as the **source** model and
the model we are moving to as the **destination** model.

**Source** Model


<img src="images/source_model.png" width="400">

**Destination Model**:


<img src="images/destination_model.png" width="400">

As we see in the illustratoins above (click to enlarge), we have many more edges in the destination than the source model, 
whiche means this is a good use case for using `pygen` for the data migration.

In this guide, we will not do the entire migration, but instead focus on the `APM_Template` to `Template` migration.

## Generating SDKs

For demo purposes we will generate the SDKs in this notebook, however, depending on your use case it might be useful to use the `pygen` CLI to generate the SDKs.

Migration scrips are often a one of, meaning you are not expected to regenerat the SDK as the model changes, thus if you generate it locally, you can do changes
to the generated code as you are not expecting this to change later. 

Note that we have set up a `config.toml` with credentials to connect to CDF

In [1]:
from cognite.pygen import load_cognite_client_from_toml, generate_sdk_notebook

In [2]:
client = load_cognite_client_from_toml()

In [3]:
source = generate_sdk_notebook(("APM_AppData_4", "APM_AppData_4", "7"), client)

Successfully retrieved data model(s) ('APM_AppData_4', 'APM_AppData_4', '7')
Writing SDK to C:\Users\ANDERS~1\AppData\Local\Temp\pygen
Done!
Added C:\Users\ANDERS~1\AppData\Local\Temp\pygen to sys.path to enable import
Imported apm_app_data_4.client


In [4]:
destination = generate_sdk_notebook(("IntegrationTestsImmutable", "ApmAppData", "v3"), client)

Successfully retrieved data model(s) ('IntegrationTestsImmutable', 'ApmAppData', 'v3')
Writing SDK to C:\Users\ANDERS~1\AppData\Local\Temp\pygen
Done!
C:\Users\ANDERS~1\AppData\Local\Temp\pygen already in sys.path
Imported apm_app_data.client


## Retrieving Data

It is common to store the nodes and edges, i.e., instances, in a differen space than the data model. To only get instances from an instance space, `sourceSpace`, 
we filter for it. In addition, it important that we retrieve all edges so we can connect the `templates` with the `templates_items`.

In [48]:
templates = source.apm_template.list(space="sourceSpace", retrieve_edges=True, limit=-1)
items = source.apm_template_item.list(space="sourceSpace", limit=-1)

In [49]:
len(templates), len(items)

(50, 105)

## Transformation

In [53]:
templates

,external_id,assigned_to,created_by_external_id,is_archived,root_location_external_id,status,title,updated_by_external_id,template_items,space,version,last_updated_time,created_time,deleted_time
0,108c5106-f638-4d9c-8ef2-4b470ed94495,None,80d32fb4-bcbc-44ae-abed-fee53b148d82,False,31d9f834-a1ab-452a-b2a5-875f0e9f384a,OnHold,good,f1e7d13d-3fd2-42e4-bebc-68ad2aa905eb,None,sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
1,32f92434-e5fb-49c1-b4c4-1a454ed699ea,None,6dbb225f-5059-4ff2-9543-7ab400e235e9,False,dac4077b-61d6-445c-a4f4-f6f3247125ab,OnHold,three,df561fd7-dad6-484e-95f7-ae967fb00ed3,None,sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
2,6b072855-8408-4c21-b1a3-5caf4bd22204,None,863a0cd1-f55e-4529-ac50-34090596a279,True,1b582c43-bc07-4ab7-959e-cf7e434711a0,Completed,also,37d17cd1-2bbe-416a-b7b1-111e44c64b36,None,sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
3,95c51139-438b-4fd8-a5ce-b80dbcd77db4,None,ac5c7d3f-564b-44d2-bdc6-6eaf23b075ce,True,1b582c43-bc07-4ab7-959e-cf7e434711a0,Completed,likely,caba29e9-fa98-438d-b5b7-6d43677764d3,None,sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
4,9544b2c9-9abe-4a9b-b106-3db9510363c2,None,b3fd00cb-8b06-4365-aac5-b0fcf6f8044c,True,41ad2ff1-88b6-4c45-9bfd-a76d0c0ba9c6,Cancelled,thing,ac5c7d3f-564b-44d2-bdc6-6eaf23b075ce,None,sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
5,21d6aa38-3157-4647-b286-009685c786d8,None,8a717d87-09c0-4b1c-abdb-721e1ff5dfa5,False,e2db2a81-5420-4252-82f5-8d85ed19385e,OnHold,wrong,b9a032ee-40ed-4cf0-bb53-d9734e0632f8,None,sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
6,5fed1620-fdaa-4c35-a530-a8d506a2bd8e,None,e20ca463-9d52-43fb-a3f2-cc793a325713,True,dac4077b-61d6-445c-a4f4-f6f3247125ab,OnGoing,our,1c945a75-17f0-4778-8dac-be1c80b73e96,None,sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
7,36afcdd3-b2e5-4ed0-af47-07ba45f50c4f,None,763a9909-e5b6-4bf3-89e3-a6c0497bd205,True,e2db2a81-5420-4252-82f5-8d85ed19385e,Cancelled,difference,ac5c7d3f-564b-44d2-bdc6-6eaf23b075ce,None,sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
8,e942481b-8abe-424d-8852-bbeb9ed53c02,None,2b0f4e82-f515-42ee-9545-03e08799c7c7,False,1ead72e6-4d6d-4e8a-bf0b-55b18875bbf4,OnGoing,heavy,ca3e9d72-1f67-4603-a770-270a7984f093,None,sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
9,0dba57ba-c183-4886-a348-9c909dfa4bb7,None,9c56ca68-8fcd-47b7-a2ef-c61661a96f3e,True,5865327a-ecb1-43b5-9818-acc7f94e353f,Completed,its,9fb9251e-0c74-4571-b0fe-e5881fa3317c,None,sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None


In [16]:
from collections import defaultdict

In [32]:
from cognite.client import data_modeling as dm

In [41]:
type_ = "node"
instances = client.data_modeling.instances.list(type_, filter=dm.filters.Equals([type_, "space"], "sourceSpace"), limit=-1)

In [42]:
len(instances)

155

In [27]:
instances

""


In [17]:
client.data_modeling.instances.delete(**{f"{type_}s": instances.as_ids()})

InstancesDeleteResult(nodes=[NodeId(space='sourceSpace', external_id='0ea95ccc-d275-4488-b25d-d86eacbcac86'), NodeId(space='sourceSpace', external_id='4795a5d7-2a06-4a8f-a61d-4493ff6a6f44'), NodeId(space='sourceSpace', external_id='78177daf-77b6-41c7-b061-d0abc9dcb8b4'), NodeId(space='sourceSpace', external_id='dedf13ea-1029-40e3-a728-00fd53c45b24'), NodeId(space='sourceSpace', external_id='4e7b9f27-1d4a-4836-a25e-b9b7391715f0'), NodeId(space='sourceSpace', external_id='56501b0e-8839-493c-97b8-1388549c3e95'), NodeId(space='sourceSpace', external_id='b1288940-a171-47f3-a305-8bd4e70a7f0f'), NodeId(space='sourceSpace', external_id='9d4d5ad8-bb1b-45a1-9620-c8411e688504'), NodeId(space='sourceSpace', external_id='4880c187-d3c9-46c2-9ad8-82bddf79b9cd'), NodeId(space='sourceSpace', external_id='7aa02e92-c93c-4c60-9fe7-131add315524'), NodeId(space='sourceSpace', external_id='75e44110-7005-4871-bcd5-0b1c2380c1e2'), NodeId(space='sourceSpace', external_id='9d8748ed-5302-4651-a570-6281713daf13'),

## Writing Data